<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Adding-a-field-to-rigid-contactors-display-files" data-toc-modified-id="Adding-a-field-to-rigid-contactors-display-files-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Adding a field to rigid contactors display files</a></span><ul class="toc-item"><li><span><a href="#Preparation" data-toc-modified-id="Preparation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Preparation</a></span><ul class="toc-item"><li><span><a href="#Simple-sample-generation" data-toc-modified-id="Simple-sample-generation-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Simple sample generation</a></span></li></ul></li><li><span><a href="#Computation" data-toc-modified-id="Computation-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Computation</a></span><ul class="toc-item"><li><span><a href="#Simulation-parameters-definition" data-toc-modified-id="Simulation-parameters-definition-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Simulation parameters definition</a></span></li><li><span><a href="#Computation-with-usual-visualization" data-toc-modified-id="Computation-with-usual-visualization-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Computation with usual visualization</a></span></li><li><span><a href="#Adding-a-field-to-Paraview" data-toc-modified-id="Adding-a-field-to-Paraview-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Adding a field to Paraview</a></span></li></ul></li><li><span><a href="#Beware:-Hacking-in-progress" data-toc-modified-id="Beware:-Hacking-in-progress-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Beware: Hacking in progress</a></span><ul class="toc-item"><li><span><a href="#Where-?" data-toc-modified-id="Where-?-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Where ?</a></span></li><li><span><a href="#What-to-do-then-?" data-toc-modified-id="What-to-do-then-?-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>What to do then ?</a></span></li></ul></li></ul></li></ul></div>

# Adding a field to rigid contactors display files


It is possible within a __pylmgc90.chipy__ python script, to add new fields to the display files generated by the `chipy.WriteDisplayFiles` function.

But depending on the file to write, the syntax may be a little cumbersome. Here will be presented a
simple case showing how to do this for the `tacts` file describing the rigid contactors.  

In the following we prepare a new sample, build a computation script and improve it to add new fields to display files. 


## Preparation

### Simple sample generation

Let's generate a simple rectangular domain filled of disks with periodic condition
as might have already be done in a previous training session (like *pre/DK_BoxJC*)


In [ ]:
from pathlib import Path
import numpy as np
from pylmgc90 import pre

datbox = Path('./DATBOX')
datbox.mkdir(exist_ok=True)

# 2D
dim = 2

# containers
bodies = pre.avatars()
mats   = pre.materials()
mods   = pre.models()
svs    = pre.see_tables()
tacts  = pre.tact_behavs()

# model
mod = pre.model(name='rigid', physics='MECAx', element='Rxx2D', dimension=dim)
mods.addModel(mod)

# materials
tdur = pre.material(name='TDURx',materialType='RIGID',density=1000.)
plex = pre.material(name='PLEXx',materialType='RIGID',density=100.)
mats.addMaterial(tdur,plex)

# generating 1000 particles with random radius between [0.5, 2.[
nb_particles = 1000
radii = pre.granulo_Random(nb_particles, 0.5, 2.)
radius_min = min(radii)
radius_max = max(radii)

# deposit in a rectangular box
lx = 75.
ly = 50. 
nb_remaining_particles, coor, radii = pre.depositInBox2D(radii, lx, ly)

for r, c in zip(radii, coor):
    body = pre.rigidDisk(r=r, center=c, model=mod, material=plex, color='BLUEx') 
    bodies += body

# lower wall
down = pre.rigidJonc(axe1=0.5*lx+radius_max, axe2=radius_max, center=[0.5*lx, -radius_max],
                     model=mod, material=tdur, color='WALLx')
down.imposeDrivenDof(component=[1, 2, 3], dofty='vlocy')
bodies += down

# interactions management:
ldkdk = pre.tact_behav(name='iqsc0',law='IQS_CLB',fric=0.3)
tacts += ldkdk
#       - avec les parois
ldkjc = pre.tact_behav(name='iqsc1',law='IQS_CLB',fric=0.5)
tacts += ldkjc

svdkdk = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='DISKx', colorCandidat   ='BLUEx', behav=ldkdk,
                       CorpsAntagoniste='RBDY2', antagoniste='DISKx', colorAntagoniste='BLUEx', alert=0.1*radius_min)
svs+=svdkdk

svdkjc = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='DISKx', colorCandidat   ='BLUEx', behav=ldkjc,
                       CorpsAntagoniste='RBDY2', antagoniste='JONCx', colorAntagoniste='WALLx', alert=0.1*radius_min)
svs+=svdkjc

# orienting gravity
g = 9.81
angle = np.pi/4
gravity = np.array( [g*np.cos(angle), -g*np.sin(angle), 0.] )


post = pre.postpro_commands()
my_command=pre.postpro_command(name='SOLVER INFORMATIONS', step=1)
post.addCommand(my_command)

# input files writing
pre.writeDatbox(dim, mats, mods, bodies, tacts, svs, post=post, gravy=gravity, datbox_path=datbox)

In [ ]:
pre.visuAvatars(bodies)

## Computation

Compared to previous examples, the example used here needs to:
1. use the `chipy.SetPeriodicCondition` function after the *initiliaze* step,
2. deactivate the default visualization step of the *computation* step.


### Simulation parameters definition

Definition of all the parameters to compute the simulation:

In [ ]:
from pylmgc90 import chipy
from pylmgc90.chipy import computation
import numpy as np

# space dimension
dim = 2

# time evolution parameters
dt = 1e-3
nb_steps = 300

# theta integrator parameter
theta = 0.5

# nlgs parameters
stype = 'Stored_Delassus_Loops       '
norm  = 'Quad ' 
conv  = 1e-4
relax = 1.0
gsit1 = 500
gsit2 = 10

# write parameter
freq_write   = 10

# display parameters
freq_display = 10

# x periodicity
lx = 75. 

### Computation with usual visualization

So, now that the parameters are all defined, an usual computation except that the display step is skipped in the generic function and done explicitely here:

In [ ]:
computation.initialize( dim, dt, theta, h5_file='lmgc90.h5' )
chipy.SetPeriodicCondition(xperiod=lx)

for k in range(1, nb_steps+1):
    if k%30 == 0:
        print( f"computing step {k}" )
    computation.one_step(stype, norm, conv, relax, gsit1, gsit2, freq_write, 0)
    chipy.WriteDisplayFiles(freq_display)

computation.finalize()
print('Computation finished')

You can check the result in paraview (if you run paraview from here,
remember to close it before trying to run new cells).

In [ ]:
#running a specific version of paraview on macos
#!/Applications/ParaView-5.3.0-RC2.app/Contents/MacOS/paraview

#running a specific version of paraview on Linux
#!paraview

### Adding a field to Paraview

In this exercice, it is desired to add the kinetic energy value
of the supporting body to all contactors.

Since all the contactor types are displayed in a single `tacts` block,
the new value must be defined for each type of contactor present in the simulation.
If any is forgotten, paraview will not display the new field.

In the end, an array to store the value to display is needed for each
contactor type. And these arrays will be stored in a dictionnary of which
keys will be the contactor name (a string).

In [ ]:
# Initialization (reading database and so on)
computation.initialize( dim, dt, theta, h5_file='lmgc90.h5' )
chipy.SetPeriodicCondition(xperiod=lx)

# Getting the number of diskx/joncx in the computation
nb_diskx = chipy.DISKx_GetNbDISKx()
nb_joncx = chipy.JONCx_GetNbJONCx()

# Defining arrays to store the kinetic energy during computation
disk_kine = np.zeros( nb_diskx )
wall_kine = np.zeros( nb_joncx )

# Dictionary definition to use when writing display files
kine_dict = { 'DISKx' : disk_kine,
              'JONCx' : wall_kine,
              }

Now, during the time loop, several steps are added:
* Get the current velocity of the bodies using the `chipy.RBDY2_GetBodyVector` function
which needs in input arguments:
  * the `V____` string to specify that it is the velocity that is needed
  * an id of rigid body (beware, numbering in Fortran starts at 1 contrary to Python/C which start at 0).
* Get the mass and inertia of the bodies using the `chipy.RBDY2_GetBodyInertia` and `chipy.RBDY2_GetBodyMass`
* Compute the kinetic energy and store it in our dedicated array.
* Remember that the velocity field is of size 3 in 2D : $[v_x, v_y, \omega_z]$
* Add an argument to the `chipy.WriteDisplayFiles` function to display our new field

In [ ]:
for k in range(1,nb_steps+1):

    if k%30 == 0:
        print( f"computing step {k}" )

    computation.one_step(stype, norm, conv, relax, gsit1, gsit2, freq_write, 0)

    
    if k%freq_display==0:    
        # for all the diskx contactors
        v = chipy.RBDY2_GetAllBodyVector('V____')
        m = chipy.RBDY2_GetAllMass()
        I = chipy.RBDY2_GetAllInertia()
        v2 = np.sum(v[:,:2]*v[:,:2],axis=1)
        #v2 = np.einsum('ij,ij->i',v[:,:2],v[:,:2])
        disk_kine = 0.5 * m[:nb_diskx] * v2[:nb_diskx] + I[:nb_diskx]*v[:nb_diskx,2]*v[:nb_diskx,2]
        # add the dictionnary to the visualization 'v_crit' is my choice
        # and is the name of the field added in paraview
        chipy.WriteDisplayFiles(1, E_kin=('tacts', kine_dict))

computation.finalize()

Now check in paraview that the `E_kin` (or any other name you choose if you changed it) field
appears in the *tacts* block files.

In [ ]:
#running a specific version of paraview on macos
#!/Applications/ParaView-5.3.0-RC2.app/Contents/MacOS/paraview

#running a specific version of paraview on Linux
#!paraview

## Beware: Hacking in progress

### Where ?
In the example above, an ugly trick is used to simplify the syntax.
The aforementionned code works only because I decided, in the pre-processing step,
to add the avatar holding the `JONCx` contactor at the very end and there is not
even one cluster of `DISKx`. If any of these previous statements were false, the code
would break.

The reason is that a rigid body (`RBDY2`) index is needed to call `chipy.RBDY2_GetBodyVector`,
but a contactor index (`DISKx`) is used. And the fact that these numbers are the same
is a specific case.


### What to do then ?

To do things properly, the user should get a mapping between each numbering tables.
This would allow to use a `for` loop on the contactor index, but get back the body
index to give to `chipy.RBDY2_GetBodyVector`.

When preparing the dictionnary to store the value to display, in addition to getting
the number of each contactor type, you need to also get the index mapping:


In [ ]:
# Initialization (reading database and so on)
computation.initialize( dim, dt, theta, h5_file='lmgc90.h5' )
chipy.SetPeriodicCondition(xperiod=lx)

# Getting the number of diskx/joncx in the computation
nb_diskx = chipy.DISKx_GetNbDISKx()
nb_joncx = chipy.JONCx_GetNbJONCx()

# Defining arrays to store the velocity during computation
disk_kine = np.zeros( nb_diskx )
wall_kine = np.zeros( nb_joncx )

# Dictionnary definition to use when writing display files
kine_dict = { 'DISKx' : disk_kine,
              'JONCx' : wall_kine,
             }

# getting the index mapping
diskx2bdyty = chipy.DISKx_GetPtrDISKx2BDYTY()

And now in the time loop, you need to use this mapping to:
* check that the `DISKx` you are considering is indeed a `RBDY2` body
* if true, use the body number to get the velocity

In [ ]:
for k in range(1, nb_steps+1):

    if k%30 == 0:
        print( f"computing step {k}" )

    computation.one_step(stype, norm, conv, relax, gsit1, gsit2, freq_write, 0)

    # for all the diskx contactors
    v = chipy.RBDY2_GetAllBodyVector('V____')
    m = chipy.RBDY2_GetAllMass()
    I = chipy.RBDY2_GetAllInertia()
    v2 = np.sum(v[:,:2]*v[:,:2],axis=1)
    #v2 = np.einsum('ij,ij->i',v[:,:2],v[:,:2])
    kine = 0.5 * m * v2 + I*v[:,2]*v[:,2]

    disk_idx = diskx2bdyty[:,2]==chipy.RBDY2_ID
    disk_idx = diskx2bdyty[disk_idx,0]-1
    disk_kine[:] = kine[disk_idx]


    chipy.WriteDisplayFiles(freq_display, E_kin=('tacts', kine_dict) ) 

computation.finalize()

In [ ]:
#!paraview

## Another field of interest

### Weber tensor

Another value of interest would be to compute the equivalent stress on each contactor using the weber tensor.

The idea in this case is first to collect the interactions and group them by involved contactor to sum the applied effort.


In [ ]:
# Initialization (reading database and so on)
computation.initialize( dim, dt, theta, h5_file='lmgc90.h5' )
chipy.SetPeriodicCondition(xperiod=lx)

# Getting the number or RBDY2 body
nb_rbdy2 = chipy.RBDY2_GetNbRBDY2()

# Getting the number of diskx/joncx in the computation
nb_diskx = chipy.DISKx_GetNbDISKx()
nb_joncx = chipy.JONCx_GetNbJONCx()

# Getting map between contactor and rigid indexing
diskx2bdyty = chipy.DISKx_GetPtrDISKx2BDYTY()
joncx2bdyty = chipy.JONCx_GetPtrJONCx2BDYTY()

# Creating some storing array
rstress = np.zeros((nb_rbdy2,2,2))
dstress = np.zeros((nb_diskx,4))
jstress = np.zeros((nb_joncx,4))

dq = np.zeros(nb_diskx)
jq = np.zeros(nb_joncx)       

dp = np.zeros(nb_diskx)
jp = np.zeros(nb_joncx)       

# Dictionary definition to use when writing display files
stress = { 'DISKx':dstress, 'JONCx':jstress } 
p = { 'DISKx':dp, 'JONCx':jp } 
q = { 'DISKx':dq, 'JONCx':jq } 

for k in range(1,nb_steps+1):

    if k%30 == 0:
        print( f"computing step {k}" )

    computation.one_step(stype, norm, conv, relax, gsit1, gsit2, freq_write, 0)
 
    if k%freq_display==0:    

        # collecting data from disk/disk and disk/jonc interactions to RBDY2
        inters = chipy.getInteractions()

        rstress[:,:,:] = 0.

        icdbdy = inters['icdbdy']-1
        ianbdy = inters['ianbdy']-1

        # projecting local reaction in global frame
        reac = np.einsum('ijk,ik->ij',inters['uc'],inters['rl'])
        #reac = np.matmul( inters['uc'], inters['rl'][:,:,np.newaxis])[:,:,0]

        # computing lever arm for candidates and antagonists
        coors  = chipy.RBDY2_GetAllBodyVector('Coor_')
        cdlev  = inters['coor'] - coors[icdbdy,:2]
        anlev  = inters['coor'] - coors[ianbdy,:2]

        # outer product (tensor product)
        cds    =  np.einsum('ij,ik->ijk', cdlev, reac )
        ans    = -np.einsum('ij,ik->ijk', anlev, reac )

        # trick to sum on each unique index of cd/an body
        np.add.at(rstress, icdbdy, cds)
        np.add.at(rstress, ianbdy, ans)

        # normalizing by area
        area   = chipy.RBDY2_GetAllArea()
        rstress/= area[:,np.newaxis,np.newaxis]

        # computing eigen values
        eigval = np.linalg.eigvals(rstress)
        rp = 0.5*(eigval[:,0]+eigval[:,1]).real
        rq = 0.5*(eigval[:,1]-eigval[:,0]).real

        # map from RBDY2 index to diskx/joncx index
        d_idx = diskx2bdyty[:,2] == chipy.RBDY2_ID
        j_idx = joncx2bdyty[:,2] == chipy.RBDY2_ID
        d_idx = diskx2bdyty[d_idx,0]-1
        j_idx = joncx2bdyty[j_idx,0]-1

        # reshape and save
        dstress[:,:] = np.reshape(rstress[d_idx,:,:], [nb_diskx,4])
        jstress[:,:] = np.reshape(rstress[j_idx,:,:], [nb_joncx,4])
        dp[:] = rp[d_idx]
        dq[:] = rq[d_idx]
        jp[:] = rp[j_idx]
        jq[:] = rq[j_idx]

        # add the dictionnary to the visualization
        chipy.WriteDisplayFiles(1, stress=('tacts',stress),p=('tacts',p), q=('tacts',q))

computation.finalize()

In [ ]:
#!paraview